In [2]:
import pandas as pd

# 데이터 불러오기, \s+ 는 공백이 하나 이상이라는 뜻
feature_name_df = pd.read_csv("../../data files/human_activity/features.txt",sep = '\s+', header = None, names = ['column_index','column_name'])

# 중복된 feature에 인덱스를 붙여 중복을 피하게 하는 함수 생성
def get_new_feature_name(old_feature_name_df):
    # cumcount 함수를 사용하여 몇번째 중복인지 확인. cumcount는 중복이 발생할경우 1씩 올라간다.
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),columns=['dup_cnt'])

    # 인덱스를 새로 추가
    feature_dup_df = feature_dup_df.reset_index()

    # feature name이 들어있는 data frame과 중복 feature 정보가 들어있는 data frame을 index 기준으로 모든 행을 병합
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(),feature_dup_df,how='outer')

    # 중복되는 feature name에 숫자 부여. 숫자는 dup_cnt에 의해 중복된 순서만큼 부여됨. 예를들어 2번째 중복이면 '_2'가 부여됨
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name','dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) if x[1] > 0 else x[0], axis =1)

    # index는 필요없으니 삭제함.
    new_feature_name_df = new_feature_name_df.drop(['index'],axis=1)
    return new_feature_name_df

# 데이터 로딩
def get_human_dataset():
    # 각 데이터파일들은 공백으로 분리되어 있기 때문에 공백문자를 sep로 할당.
    feature_name_df = pd.read_csv('../../data files/human_activity/features.txt',sep='\s+',header=None,names=['column_index','column_name'])
    
    # 중복된 feature명을 수정
    new_feature_name_df = get_new_feature_name(feature_name_df)

    # data frame에 피처명을 컬럼으로 부여하기위해 리스트 객체로 다시 변환 모든 feature name은 1번째 열에 있음.
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()

    # 학습 feature 데이터 셋과 테스트 feature 데이터를 dataframe으로 로딩. 컬럼은  feature_name 적용
    x_train = pd.read_csv('../../data files/human_activity/train/X_train.txt',sep='\s+',names=feature_name)
    x_test = pd.read_csv('../../data files/human_activity/test/X_test.txt',sep='\s+',names=feature_name)

    # 학습 label과 테스트 label 데이터를 data frame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('../../data files/human_activity/train/y_train.txt',sep='\s+',names=['action'])
    y_test = pd.read_csv('../../data files/human_activity/test/y_test.txt',sep='\s+',names=['action'])

    # 로드된 학습/테스트용 데이터프레임을 모두 반환
    return x_train,x_test,y_train,y_test

# 데이터 준비
x_train, x_test, y_train, y_test = get_human_dataset()

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측성능평가
rf_clf = RandomForestClassifier(random_state=56)
rf_clf.fit(x_train,y_train)
pred_rf = rf_clf.predict(x_test)
accuracy = accuracy_score(y_test,pred_rf)
print("Random Forest Accuracy : {:.4f}".format(accuracy))
rf_clf.get_params()

Random Forest Accuracy : 0.9233


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 56,
 'verbose': 0,
 'warm_start': False}

In [11]:
# 하이퍼 파라미터 설정
from sklearn.model_selection import GridSearchCV

params = {'n_estimators' : [100],
          'max_depth' : [6,8,10,12,14,16,18,20], 
          'min_samples_leaf' : [1,2,4,6,8,12,18,24], 
          'min_samples_split' : [8,16,20,24]
          }

# random forest classifier 객체 생성 후 grid search cv 수행, n_jobs = -1 로 설정하면 모든 cpu의 코어를 사용하여 속도가 빨라짐.
rf_clf = RandomForestClassifier(random_state=56)
grid_cv = GridSearchCV(rf_clf,param_grid=params, cv=3)
grid_cv.fit(x_train,y_train)

print("최적 하이퍼 파라미터 : ",grid_cv.best_params_)
print("최고 예측 정확도 {:.4f}".format(grid_cv.best_score_))

최적 하이퍼 파라미터 :  {'max_depth': 14, 'min_samples_leaf': 18, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도 0.9199
